## `facebook/bart-large-cnn` 
### `pipeline`

In [1]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

2022-06-14 08:48:48.790156: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-14 08:48:49.433280: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10794 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


In [2]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


### `fine tuning this model on my data`

In [1]:
import pandas as pd

In [2]:
data_file = r'response-dialog-sum-all.json'
df = pd.read_json(data_file).drop(columns=['id'])
df = df[:1000]

In [3]:
data = pd.read_json('response-sum.json').drop('id',axis=1)
data.head()

,summ,dialogue
0,manuj wanted to know about the renewal of his ...,prashanth: welcome to celebal tech ivr automat...
1,manuj has made a transaction of 42000 on flipk...,prashanth: welcome to celebal technologies ivr...
2,manuj is not interested in emi option service ...,"prashanth: hi, i am prashanth calling you from..."
3,manuj has made a transaction of 35000 on amazo...,"prashanth: hi, i'm prashanth calling you from ..."
4,manuj wanted to know bout his loan application...,prashanth: welcome to celebal tech bank. manuj...


In [4]:
data_new = data.append(df)

/tmp/ipykernel_25934/2877896904.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_new = data.append(df)


In [15]:
data_new.shape

(1020, 2)

In [16]:
data_new.head()

,summ,dialogue
0,manuj wanted to know about the renewal of his ...,prashanth: welcome to celebal tech ivr automat...
1,manuj has made a transaction of 42000 on flipk...,prashanth: welcome to celebal technologies ivr...
2,manuj is not interested in emi option service ...,"prashanth: hi, i am prashanth calling you from..."
3,manuj has made a transaction of 35000 on amazo...,"prashanth: hi, i'm prashanth calling you from ..."
4,manuj wanted to know bout his loan application...,prashanth: welcome to celebal tech bank. manuj...


In [4]:
# df.columns = ['summary','text']
# df = df.dropna()
# df.head()

,summary,text
0,"mr. smith's getting a check-up, and doctor haw...","person1: hi, mr. smith. i'm doctor hawkins. wh..."
1,mrs parker takes ricky for his vaccines. dr. p...,"person1: hello mrs. parker, how have you been?..."
2,person1's looking for a set of keys and asks f...,"person1: excuse me, did you see a set of keys?..."
3,person1's angry because person2 didn't tell pe...,person1: why didn't you tell me you had a girl...
4,malik invites nikki to dance. nikki agrees if ...,"person1: watsup, ladies! y'll looking'fine ton..."


In [5]:
# df = df[:1000]

In [18]:
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(df,test_size=0.15)
train_df.shape,test_df.shape

((850, 2), (150, 2))

In [19]:
import datasets
import pandas as pd
from fastai.text.all import *
#from transformers import *

from blurr.text.data.all import *
from blurr.text.modeling.all import *

import nltk
nltk.download('punkt', quiet=True)

True

In [20]:
import transformers
from transformers import BartForConditionalGeneration

In [21]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

In [22]:
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization',
    text_gen_kwargs=
 {'max_length': 248,'min_length': 56,'do_sample': False, 'early_stopping': True, 'num_beams': 4, 'temperature': 1.0, 
  'top_k': 50, 'top_p': 1.0, 'repetition_penalty': 1.0, 'bad_words_ids': None, 'bos_token_id': 0, 'pad_token_id': 1,
 'eos_token_id': 2, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1, 'decoder_start_token_id': 2, 'use_cache': True, 'num_beam_groups': 1,
 'diversity_penalty': 0.0, 'output_attentions': False, 'output_hidden_states': False, 'output_scores': False,
 'return_dict_in_generate': False, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2, 'remove_invalid_values': False})


In [23]:
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
blocks

(<blurr.text.data.seq2seq.core.Seq2SeqTextBlock at 0x7f7f1fd74760>,
 <function fastai.imports.noop(x=None, *args, **kwargs)>)

In [25]:
dblock = DataBlock(blocks=blocks, get_x=ColReader('dialogue'), get_y=ColReader('summ'), splitter=RandomSplitter())
dls = dblock.dataloaders(df, batch_size = 16)

In [26]:
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'en' },
            'returns': ["precision", "recall", "f1"]
        }
    }

In [27]:
# !pip install bert_score

In [28]:
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

In [ ]:
learn.fit_one_cycle(10, lr_max=3e-5, cbs=fit_cbs)

/home/harshvardhan-pathak/Workspace/Customer_analytics/canalytics_env/lib/python3.8/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/home/harshvardhan-pathak/Workspace/Customer_analytics/canalytics_env/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
learn.show_results(learner=learn, max_n=2)

In [ ]:
test_article = """
The past 12 months have been the worst for aviation fatalities so far this decade - with the total of number of people killed if airline 
crashes reaching 1,050 even before the Air Asia plane vanished. Two incidents involving Malaysia Airlines planes - one over eastern Ukraine and the other in the Indian Ocean - led to the deaths of 537 people, while an Air Algerie crash in Mali killed 116 and TransAsia Airways crash in Taiwan killed a further 49 people. The remaining 456 fatalities were largely in incidents involving small commercial planes or private aircraft operating on behalf of companies, governments or organisations. Despite 2014 having the highest number of fatalities so far this decade, the total number of crashes was in fact the lowest since the first commercial jet airliner took off in 1949 - totalling just 111 across the whole world over the past 12 months. The all-time deadliest year for aviation was 1972 when a staggering 2,429 people were killed in a total of 55 plane crashes - including the crash of Aeroflot Flight 217, which killed 174 people in Russia, and Convair 990 Coronado, which claimed 155 lives in Spain. However this year's total death count of 1,212, including those presumed dead on board the missing Air Asia flight, marks a significant rise on the very low 265 fatalities in 2013 - which led to it being named the safest year in aviation since the end of the Second World War. Scroll down for videos. Deadly: The past 12 months have been the worst for aviation fatalities so far this decade - with the total of number of people killed if airline crashes reaching 1,158 even before the Air Asia plane (pictured) vanished. Fatal: Two incidents involving Malaysia Airlines planes - one over eastern Ukraine (pictured) and the other in the Indian Ocean - led to the deaths of 537 people. Surprising: Despite 2014 having the highest number of fatalities so far this decade, the total number of crashes was in fact the lowest since the first commercial jet airliner took off in 1949. 2014 has been a horrific year for Malaysia-based airlines, with 537 people dying on Malaysia Airlines planes, and a further 162 people missing and feared dead in this week's Air Asia incident. In total more than half the people killed in aviation incidents this year had been flying on board Malaysia-registered planes. In January a total of 12 people lost their lives in five separate incidents, while the same number of crashes in February killed 107. 
"""

In [ ]:
print('The past 12 months have been the worst for aviation fatalities so far this decade - with the total of number of people killed if airline crashes reaching 1,050 even before the Air Asia plane vanished. Two incidents involving Malaysia Airlines planes - one over eastern Ukraine and the other in the Indian Ocean - led to the deaths of 537 people, while an Air Algerie crash in Mali killed 116 and TransAsia Airways crash in Taiwan killed a further 49 people. The remaining 456 fatalities were largely in incidents involving small commercial planes or private aircraft operating on behalf of companies, governments or organisations. Despite 2014 having the highest number of fatalities so far this decade, the total number of crashes was in fact the lowest since the first commercial jet airliner took off in 1949 - totalling just 111 across the whole world over the past 12 months. The all-time deadliest year for aviation was 1972 when a staggering 2,429 people were killed in a total of 55 plane crashes - including the crash of Aeroflot Flight 217, which killed 174 people in Russia, and Convair 990 Coronado, which claimed 155 lives in Spain. However this years total death count of 1,212, including those presumed dead on board the missing Air Asia flight, marks a significant rise on the very low 265 fatalities in 2013 - which led to it being named the safest year in aviation since the end of the Second World War. Scroll down for videos. Deadly: The past 12 months have been the worst for aviation fatalities so far this decade - with the total of number of people killed if airline crashes reaching 1,158 even before the Air Asia plane (pictured) vanished. Fatal: Two incidents involving Malaysia Airlines planes - one over eastern Ukraine (pictured) and the other in the Indian Ocean - led to the deaths of 537 people. Surprising: Despite 2014 having the highest number of fatalities so far this decade, the total number of crashes was in fact the lowest since the first commercial jet airliner took off in 1949. 2014 has been a horrific year for Malaysia-based airlines, with 537 people dying on Malaysia Airlines planes, and a further 162 people missing and feared dead in this weeks Air Asia incident. In total more than half the people killed in aviation incidents this year had been flying on board Malaysia-registered planes. In January a total of 12 people lost their lives in five separate incidents, while the same number of crashes in February killed 107.') 

In [ ]:
outputs = learn.blurr_summarize(test_article, early_stopping=True, num_beams=4, num_return_sequences=3)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

In [ ]:
learn.metrics = None
learn.export(fname='ft_cnndm_export.pkl')

In [ ]:
inf_learn = load_learner(fname='ft_cnndm_export.pkl')
inf_learn.blurr_summarize(test_article)

# On our data

In [ ]:
data_file = r'our_data.json'
df = pd.read_json(data_file)

In [ ]:
df.head()

In [ ]:
summ = []
for i in df.dialogue:
    summary = inf_learn.blurr_summarize(i)
    summ.append(summary)

In [15]:
df['prediction'] = summ

In [16]:
df.head()

,id,summ,dialogue,prediction
0,10,"manuj wanted to know about the renewal of his policy. his mobile number is 9998887776, date of birth is 25th august 2021, prashanth has send him the renewal payemt link.",prashanth: welcome to celebal tech ivr automation demo. hi how are you? manuj: i'm fine. prashanth: thank you for calling city health insurance my name is prashanth. how can i help you today? manuj: i have an health policy with you. i wanted to know do i need to renew the policy?\nprashanth: of course i can do that. could you please confirm few details before i give you the necessary information? could you please tell me your registered mobile number? manuj: yeah sure it's 9998887776. i can confirm that you have active health insurance policy with us. can you confirm your name and date of ...,[{'summary_texts': ' manuj calls city health insurance to renew his health insurance policy. prashanth will send him a payment link of renewal for the 2 year option. manuj will receive the payment link on his registered mobile number and email address and will share the link with him shortly.'}]
1,12,"manuj has made a transaction of 42000 on flipkart on 16th oct 2021 through your credit card, he is ready to pay full payment.","prashanth: welcome to celebal technologies ivr automation demo. i can help with bank related queries. manuj:hello. prashanth:hello, i'm prashanth calling from celebal bank. am i talking to mr manuj? manuj:yes you are. prashanth:thanks for confirming your name mr manuj. this call is being recorded by celebal tech bank. for security reasons, please do not share your cvv and otp with anyone.\nmanuj: okay. prashanth:i can see that you have recently done the transaction of 42,000 from flipkart on 16th oct 2021 through your credit card. you can instantly get this amount converted into emi. would...","[{'summary_texts': ' mr manuj calls prashanth from celebal bank to ask about converting the 42,000 from flipkart transaction into emi. by opting for emi, manuj won't block a big amount and it won't affect his monthly budget.'}]"
2,13,manuj is not interested in emi option service and he wanted to talk to an agent,"prashanth: hi, i am prashanth calling you from sellable check bank. we have and emi offer specially created for you. would you like to know more about the offers? manuj: no i am not interested. prashanth: i respect that, sir, but by opting for an emi you are not blocking huge cash which makes your personal budgeting for the month easier. you can also get to know precisely how much money you need to pay towards the emi each month. would you like to know about the offer? manuj: no , i wolud like to talk to your agent. prashanth: please wait while we are transferring your call to our agent.",[{'summary_texts': ' manuj calls prashanth from sellable check bank to talk about an emi offer specially created for manuj. prashant tells manuj that by opting for the emi you are not blocking huge cash which makes budgeting for the month easier. manuj prefers to talk to the agent.'}]
3,14,manuj has made a transaction of 35000 on amazona and he is ready to full payment.,"prashanth: hi, i'm prashanth calling you from celebal tech bank. we have and emi offer specially created for you. would you like to know more about the offer?\nmanuj: yes sure.\nprashanth: this call is being recorded by celebal tech bank. for security reasons, please do not share your cvv and otp with anyone. as i can see a transaction of 35,000 done on amazon. would you like to convert this into emi? manuj:yes, convert this one. prashanth :that's a great choice, sir. could you please confirm at what tenure would you be interested in?\nmanuj: tell me the monthly payment for the tenure of 6...","[{'summary_texts': ' prashanth calls manuj from celebal tech bank to suggest an emi offer specially created for manuj to convert a transaction of 35,000 done on amazon into emi. manuj is happy to pay full payment and will keep banking with celebal bank.'}]"
4,15,"manuj want

In [17]:
df.to_csv('with_prediction.csv')

In [18]:
dicts = {}
lists = []
for i in range(len(df)):
    dicts = {"id":i,
             "summary":df.summ[i],
             "prediction":df.prediction[i]
            }
    lists.append(dicts)

In [19]:
for i in lists:
    print(i)
    print('--'*50)
    print('--'*50)
    print('--'*50)

{'id': 0, 'summary': 'manuj wanted to know about the renewal of his policy. his mobile number is 9998887776, date of birth is 25th august 2021, prashanth has send him the renewal payemt link.', 'prediction': [{'summary_texts': ' manuj calls city health insurance to renew his health insurance policy. prashanth will send him a payment link of renewal for the 2 year option. manuj will receive the payment link on his registered mobile number and email address and will share the link with him shortly.'}]}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
{'id': 1, 'summary': 'manuj has made a transaction of 42000 on flipkart on 16th oct 2021 through your credit card, he is ready to pay full payment.', 'prediction': [{'summary_texts': " mr man

## `Integration -------------------------------------------------`

In [5]:
import docx

In [6]:
docx

<module 'docx' from '/anaconda/envs/py38_pytorch/lib/python3.8/site-packages/docx.py'>

In [24]:
import docx.document

ModuleNotFoundError: No module named 'docx.document'; 'docx' is not a package

In [22]:
import docx
def preprocess_transcript(doc_path):
    doc = docx.Document(doc_path)
    speakers = []
    resp = []
    new_paras = [p.text for p in doc.paragraphs if p.text]
    speaker_map = {'Agent': None, 'Customer': None}
    for n,i in enumerate(new_paras):
        a = i.split('\n')
        if n==0:
            speaker_map['Agent'] = a[1]
            
        new_paras_better = f"{a[2]}".replace("Kashetti Prashanth", "Prashanth").replace("Manuj Kumar Joshi", "Manuj")
        resp.append([ a[1], process_timestamp(a[0]), new_paras_better])
        
    return resp, speaker_map

In [23]:
doc_text_with_ts, speaker_map = preprocess_transcript('21.docx')

AttributeError: module 'docx' has no attribute 'Document'

In [60]:
def extract_text(doc_text_with_ts):
    text = []
    for i in doc_text_with_ts:
        text.append(f"{i[0]}: {i[2]}")
    return preprocess(' '.join(map(str, text)))

In [57]:
txt = extract_text(doc_text_with_ts)

NameError: name 'doc_text_with_ts' is not defined